# Define Cnfigruations

In [585]:
# Will ignore everything else, only focus on the columns of interest
#columns_of_interest=['Team', 'Number', 'Position', 'Age', 'Height', 'Weight', 'Salary', 'Some Error']
columns_of_interest=None

# Works reversely as columns_of_interest, ignore those columns. 
# It should not use together with column of interest
columns_to_ignore=['College']

# Dara source url
data_src = "https://media.geeksforgeeks.org/wp-content/uploads/nba.csv"

# Foce certian columns data type to be number
force_to_number = [{'column':'Height', 'method':'replace_dash'}]

The force_to_number configuration can be a bit tricky. If a data set that contains not properly formatted values, I think we can provide some gereic function help user force the value to be numeric value. 
E.g. in the provided data, the value "Height" is saperated by dash '-' ...

But it doesn't work this way, the height have to be converted to centmeters in order to get the proper numeric valye. This way is wrong. 

# Read Data

In [586]:
import pandas as pd

pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_rows', None)

data = pd.read_csv(data_src) 

In [587]:
# Preview the data
pd.options.display.max_columns = 100 # Set max dispaly columns, if the data has too many columns
data.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


# Pre-process Data

### Force data to be number

In [588]:
# Old methods, no in use
def replace_dash(s):
	if ( isinstance(s, str)):
		n = s.replace('-','.')
		n = float(n)
		return n
	else:
		return s

forceMethods = {
	'replace_dash': replace_dash
}


In [589]:
# Old method to resolve the '-' in height

#if(force_to_number is not None):
	#for force_item in force_to_number:
		#column = force_item['column']
		#method = force_item['method']
		#force_func = forceMethods[method]
		#data[column] = data[column].apply(func=force_func)

In [590]:
def feet_inch_to_cm(s):
	if (isinstance(s, str)):
		feet = s.split('-')[0]	
		inch = s.split('-')[1]
		cm = float(feet) * 30.4800 + float(inch) * 2.54
		return cm
	else:
		return s

In [591]:

data['Height'] = data['Height'].apply(feet_inch_to_cm)

data.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,187.96,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,198.12,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,195.58,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,195.58,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,208.28,231.0,NaN,5000000.0


### Focus on the columns of interest

In [592]:
# A container for all the error-messages in the description
missing_columns_of_interests = [] 

# Check if there is error in conlums_of_interest, if user interested
# in a column that doesn't exist, record that to generate error message
if(columns_of_interest is not None):
	for column in columns_of_interest:
		if not column in data.columns:
			missing_columns_of_interests.append(column)
	
	# Loop through all columns in data frame, drop those columns that is
	# not interested
	for column in data.columns:
		if not column in columns_of_interest:
			data.drop(column, inplace=True, axis=1)



missing_columns_of_interests

[]

In [593]:
# Ignore the columns to ignore
if (columns_to_ignore is not None):
	for column in columns_to_ignore:
		if column in data.columns:
			data.drop(column, inplace=True, axis=1)

# Overall data description

In [594]:
# Count rows and columns
data_shape = data.shape
total_rows_num = data_shape[0]
total_columns_num = data_shape[1]

print(total_rows_num, total_columns_num)

458 8


In [595]:
# Count total rows that contais a missed value
incomplete_rows = data[data.isna().any(axis=1)]
incomplete_rows_num = incomplete_rows.shape[0]

In [596]:
# Count percentage of incomplete rows
incomplete_rows_pct = incomplete_rows_num/total_rows_num
incomplete_rows_pct

0.026200873362445413

### Calculate missing data for each column
Only calculate the colums set in the columns_of_interest

In [597]:
missing_data = data.isna()
missing_data.head()

,Name,Team,Number,Position,Age,Height,Weight,Salary
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False


In [598]:
missing_data_num = missing_data.sum()
missing_data_sum = missing_data_num/len(data)

In [599]:
desc_idx = ['Missing Data (%)','Data Type' ,'Mean', 'Median', 'Min', 'Max', 'Most Appeared']
desc = pd.DataFrame(index=desc_idx)

for k, v in missing_data_sum.items():
	desc.at['Missing Data (%)', k] = round(v,4) * 100

desc

,Name,Team,Number,Position,Age,Height,Weight,Salary
Missing Data (%),0.22,0.22,0.22,0.22,0.22,0.22,0.22,2.62
Data Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Median,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Most Appeared,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Columte the mean and median of each column

In [600]:
# Data type of each column
for k, v in data.dtypes.items():
	desc.at['Data Type', k] = v

# Mean value
for k, v in data.mean(skipna=True, numeric_only=True).items():
	desc.at['Mean', k] = round(v, 4)

# Median
for k, v in data.median(skipna=True, numeric_only=True).items():
	desc.at['Median', k] = round(v, 4)

# Min value
for k, v in data.min(skipna=True, numeric_only=True).items():
	desc.at['Min', k] = round(v, 4)

# Max value
for k, v in data.max(skipna=True, numeric_only=True).items():
	desc.at['Max', k] = round(v, 4)

# Most frequent value
data_mode = data.mode()
for k in data_mode:
	v = data_mode[k][0]
	desc.at['Most Appeared', k] = v
	w = data[k].value_counts()[v]
	desc.at['Most Appeared Count', k] = w

# Count most frequent value



desc

,Name,Team,Number,Position,Age,Height,Weight,Salary
Missing Data (%),0.22,0.22,0.22,0.22,0.22,0.22,0.22,2.62
Data Type,object,object,float64,object,float64,float64,float64,float64
Mean,NaN,NaN,17.6783,NaN,26.9387,201.1435,221.523,4842684.1054
Median,NaN,NaN,13.0,NaN,26.0,203.2,220.0,2839073.0
Min,NaN,NaN,0.0,NaN,19.0,175.26,161.0,30888.0
Max,NaN,NaN,99.0,NaN,40.0,220.98,307.0,25000000.0
Most Appeared,Aaron Brooks,New Orleans Pelicans,5.0,SG,24.0,205.74,220.0,947276.0
Most Appeared Count,1,19,23,102,47,59,29,31


In [601]:
# removing null values to avoid errors 
# data.dropna(inplace = True) 

# percentile list
# perc =[.20, .40, .60, .80]

# list of dtypes to include
# include =['object', 'float', 'int']

# calling describe method
# desc = data.describe(percentiles = perc, include = include)

# Render the description

In [602]:
from tabulate import tabulate

print('Incomplete rows(%):'.ljust(30), round(incomplete_rows_pct, 4) * 100)
print('Total rows:'.ljust(30), total_rows_num)
print('Incomplete rows cound'.ljust(30), incomplete_rows_num)
print('Total columns of interest:'.ljust(30), total_columns_num )
print('-'*40)
print('Data description:')
print(tabulate(desc, headers='keys', tablefmt='tsv'))
print('-'*40)
print('Incomplete rows')
print(tabulate(data[data.isnull().any(axis=1)],headers='keys', tablefmt='tsv', maxcolwidths=[999, 999]))

Incomplete rows(%):            2.62
Total rows:                    458
Incomplete rows cound          12
Total columns of interest:     8
----------------------------------------
Data description:
                   	Name        	Team                	Number  	Position  	Age    	Height  	Weight  	Salary
Missing Data (%)   	0.22        	0.22                	0.22    	0.22      	0.22   	0.22    	0.22    	2.62
Data Type          	object      	object              	float64 	object    	float64	float64 	float64 	float64
Mean               	nan         	nan                 	17.6783 	nan       	26.9387	201.1435	221.523 	4842684.1054
Median             	nan         	nan                 	13.0    	nan       	26.0   	203.2   	220.0   	2839073.0
Min                	nan         	nan                 	0.0     	nan       	19.0   	175.26  	161.0   	30888.0
Max                	nan         	nan                 	99.0    	nan       	40.0   	220.98  	307.0   	25000000.0
Most Appeared      	Aaron Brooks	New Orle